In [8]:
!pip install crawl4ai
!pip install nest_asyncio
!playwright install
!pip install tavily-python


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Please install them with the following command:      ║
║                                                      ║
║     sudo playwright install-deps                     ║
║                                                      ║
║ Alternatively, use apt:                              ║
║     sudo apt-get install libevent-2.1-7t64           ║
║                                                      ║
║ <3 Playwright Team                                   ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/home/prakhar/Documents/strollr/crawlai/env/lib/python3.11/site-packages/playwri

In [2]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 1.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
from tavily import TavilyClient
import os
# import google.generativeai as genai

# model = genai.GenerativeModel("gemini-1.5-pro-latest")
# prompt = """List a few popular cookie recipes in JSON format.

# Use this JSON schema:

# Recipe = {'recipe_name': str, 'ingredients': list[str]}
# Return: list[Recipe]"""
# result = model.generate_content(prompt)
# print(result)


# Step 1. Instantiating your TavilyClient
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

# Step 2. Executing a simple search query
response = tavily_client.search("kids activities in sydney")

# Step 3. That's it! You've done a Tavily Search!
print(response)

{'query': 'kids activities in sydney', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Things to Do with Kids in Sydney | Family-Friendly Fun - Time Out', 'url': 'https://www.timeout.com/sydney/kids', 'content': "Your guide to Sydney's best things to do with kids, family-friendly restaurants, kids' shows, kids' shops and more. ... Activities for kids in Sydney. Kids. The best playgrounds in Sydney.", 'score': 0.9992592, 'raw_content': None}, {'title': '26 Of The Best Things To Do With Kids In Sydney - Urban List', 'url': 'https://www.theurbanlist.com/sydney/a-list/things-to-do-with-kids-sydney', 'content': "Best Adventure Activities With Kids In Sydney BridgeClimb Sydney 3 Cumberland Street, The Rocks. Image credit: BridgeClimb Sydney | Instagram . Kids over the age of eight and in need of something a little more exciting than Lego can join the adults on the famous BridgeClimb Sydney. Ascend as a family up the Harbour Bridge (aka the world's", 'score':

In [2]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [4]:
import typing_extensions as typing
import google.generativeai as genai
from config import Settings

settings = Settings()


async def get_final_structured_data_from_content():
    genai.configure(api_key=settings.OPENAI_API_KEY)

    # class OutputFormat(typing.TypedDict):
    #     business_name: str
    #     address: str
    #     phone_number: str
    #     email: str
    #     website_link: str

    prompt = """
    Say Hi
    """

    model = genai.GenerativeModel("gemini-1.5-pro-latest")
    result = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            # response_schema=list[OutputFormat],
            temperature=0.7,
            candidate_count=1,
            max_output_tokens=1024,
        ),
    )

    print(result.text)
    return result.text


In [3]:
import os
import asyncio
from tavily import TavilyClient
from crawl4ai import AsyncWebCrawler, CacheMode
import google.generativeai as genai

import typing_extensions as typing


async def query_generation(location: str, postcode: str):
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

    keywords = [
        "Arts & Crafts",
        "Book Babies",
        "Dance Classes",
        "Events",
        "Gymnastics",
        "Learn Languages",
        "Mums n Bubs Fitness Classes",
        "Music Classes",
        "Playgroup",
        "Playroom",
        "Rhyme Time",
        "Sensory Classes",
        "Singing Lessons",
        "Swimming Lessons",
        "Toddler Time",
        "Other",
    ]
    venue = [
        "Aquatic Centre",
        "Library",
        "Museum",
        "Play Centre",
        "Playground",
        "Toy Library",
        "Other",
        "Community Centre",
    ]

    class OutputFormat(typing.TypedDict):
        query_number: int
        query: str

    prompt = """Generate 20 different search queries to find websites, listings, and information about kids activities in {location} with postcode {postcode}. 
    Each query should combine activities from this list: {keywords}
    with venues from this list: {venue}
    
    Follow these rules for effective search queries:
    - Include specific Sydney locations or suburbs when relevant
    - Add search-relevant terms like "classes", "schedule", "booking", "programs"
    - Include age groups (toddler, preschool, kids) where appropriate
    - Make queries specific enough to find actual programs, not general information
    - Format: "primary activity/topic" + "location/venue" + "relevant qualifiers"
    
    Examples:
    - "toddler swimming lessons aquatic centre eastern suburbs sydney schedule"
    - "kids art classes community centres inner west sydney bookings"
    """.format(keywords=keywords, venue=venue, location=location, postcode=postcode)

    model = genai.GenerativeModel("gemini-1.5-pro-latest")
    result = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema=list[OutputFormat],
            temperature=0.7,
            candidate_count=1,
            max_output_tokens=1024,
        ),
    )

    print("query", result.text)  # Print the generated queries
    return result


async def get_relevant_urls(query: str):
    # Step 1. Instantiating your TavilyClient
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

    # Step 2. Executing a simple search query
    response = tavily_client.search(query)

    # Step 3. That's it! You've done a Tavily Search!
    print("response tavily", response)
    return response


async def crawl_dynamic_content():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url="https://www.timeout.com/sydney/kids",
            # Content filtering
            word_count_threshold=10,
            excluded_tags=["form", "header", "nav", "footer", "aside"],
            exclude_external_links=False,
            # Content processing
            process_iframes=False,
            remove_overlay_elements=True,
            # Cache control
            cache_mode=CacheMode.ENABLED,  # Use cache if available
            magic=True,  # Enable all anti-detection features
            # simulate_user=True,  # Simulate human behavior
            # override_navigator=True,  # Override navigator properties
        )

        content = None

        if result.success:
            content = {
                # "title": result.title,
                "url": result.url,
                "content": result.markdown,
                "images": result.media["images"],
                "links": result.links["internal"],
                "external_links": result.links["external"],
            }

            print(content)

            return content

        # if result.success:
        #     # Print clean content
        #     print("Content:", content)  # First 500 chars

        #     # Process images
        #     for image in result.media["images"]:
        #         print(f"Found image: {image['src']}")
        #         content["images"].append(image["src"])

        #     # Process links
        #     for link in result.links["internal"]:
        #         print(f"Internal link: {link['href']}")

        else:
            print(f"Crawl failed: {result.error_message}")
            return {"error": result.error_message}


async def main():
    # return await crawl_dynamic_content()
    postcode = input("Enter postcode: ")
    location = input("Enter location: ")
    query = await query_generation(location=location, postcode=postcode)

    urls = []

    for q in query:
        urls = await get_relevant_urls(q["query"])
        urls.extend(urls)

    print("final urls", urls)


if __name__ == "__main__":
    asyncio.run(main())


query [{"query": "Arts & Crafts classes Play Centre 2839 NSW", "query_number": 1}, {"query": "Book Babies sessions Library Griffith NSW 2839", "query_number": 2}, {"query": "Kids Dance Classes Community Centre near 2839", "query_number": 3}, {"query": "Children's Events Playground Griffith NSW 2839", "query_number": 4}, {"query": "Gymnastics classes for toddlers near 2839 NSW", "query_number": 5}, {"query": "Learn Languages programs Library 2839 postcode", "query_number": 6}, {"query": "Mums n Bubs Fitness Classes Griffith 2839 booking", "query_number": 7}, {"query": "Music Classes for preschoolers near postcode 2839", "query_number": 8}, {"query": "Playgroup sessions Community Centre 2839 NSW", "query_number": 9}, {"query": "Playroom activities Toy Library near Griffith 2839", "query_number": 10}, {"query": "Rhyme Time schedule Library 2839 NSW", "query_number": 11}, {"query": "Sensory Classes for babies near 2839", "query_number": 12}, {"query": "Singing Lessons for kids Griffith 283

/usr/lib/python3.11/re/__init__.py:258: RuntimeWarning: coroutine 'main' was never awaited
  return pattern.translate(_special_chars_map)


TypeError: 'GenerateContentResponse' object is not subscriptable

In [11]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [12]:
import asyncio
from crawl4ai import AsyncWebCrawler, CacheMode


async def main():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url="https://www.timeout.com/sydney/kids",
            # Content filtering
            word_count_threshold=10,
            excluded_tags=["form", "header", "nav", "footer", "aside"],
            exclude_external_links=False,
            # Content processing
            process_iframes=False,
            remove_overlay_elements=True,
            # Cache control
            cache_mode=CacheMode.ENABLED,  # Use cache if available
            magic=True,  # Enable all anti-detection features
            # simulate_user=True,  # Simulate human behavior
            # override_navigator=True,  # Override navigator properties
        )

        if result.success:
            # Print clean content
            print("Content:", result.markdown)  # First 500 chars

            # Process images
            for image in result.media["images"]:
                print(f"Found image: {image['src']}")

            # Process links
            for link in result.links["internal"]:
                print(f"Internal link: {link['href']}")

        else:
            print(f"Crawl failed: {result.error_message}")


if __name__ == "__main__":
    asyncio.run(main())

[INIT].... → Crawl4AI 0.4.1
[FETCH]... ↓ https://www.timeout.com/sydney/kids... | Status: True | Time: 0.01s
[SCRAPE].. ◆ Processed https://www.timeout.com/sydney/kids... | Time: 11ms
[COMPLETE] ● https://www.timeout.com/sydney/kids... | Status: True | Total: 0.02s
Content: [](/sydney/kids/school-holidays-in-sydney)

##### Kids

### [The best things to do with kids in Sydney](/sydney/kids/school-holidays-in-sydney)

Our ultimate list of the best kid-friendly activities and experiences Sydney, including indoor activities for wet days.

[](/sydney/kids/the-best-playgrounds-in-sydney)

##### Kids

### [The best playgrounds in Sydney](/sydney/kids/the-best-playgrounds-in-sydney)

Got a wriggly little one? Let them run off some of that energy at these parks and playgrounds.

[](/sydney/kids/the-best-kid-friendly-pubs-in-sydney)

##### Bars

### [The best kid-friendly pubs in Sydney](/sydney/kids/the-best-kid-friendly-pubs-in-sydney)

We've rounded up the Sydney pubs that have the best facil

In [12]:
import asyncio
# from crawl4ai import AsyncWebCrawler


# async def simple_crawl():
#     async with AsyncWebCrawler() as crawler:
#         result = await crawler.arun(
#             url="https://www.nbcnews.com/business",
#             bypass_cache=True,  # By default this is False, meaning the cache will be used
#         )
#         print(result.markdown[:500])  # Print the first 500 characters
from crawl4ai import AsyncWebCrawler, CacheMode  # Add CacheMode import


async def simple_crawl():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://webscraper.io/test-sites",
            cache_mode=CacheMode.BYPASS,  # Updated from bypass_cache=True
        )
        print("result", result.markdown[:500])


asyncio.run(simple_crawl())

result Toggle navigation

[![Web Scraper](/img/logo_white.svg)](/)

  * [ Web Scraper ](/)
  * [ Cloud Scraper ](/cloud-scraper)
  * [ Pricing ](/pricing)
  * Learn

    * [Documentation](/documentation)
    * [Video Tutorials](/tutorials)
    * [How to](/how-to-videos)
    * [Test Sites](/test-sites)
    * [Forum](https://forum.webscraper.io/)
  * [Install](https://chromewebstore.google.com/detail/web-scraper/jnhgnonknehpejjnehehllkliplmbmhn?hl=en)
  * [Cloud Login](https://cloud.webscraper.io/)



# 


In [8]:
async def crawl_dynamic_content():
    # You can use wait_for to wait for a condition to be met before returning the result
    # wait_for = """() => {
    #     return Array.from(document.querySelectorAll('article.tease-card')).length > 10;
    # }"""

    # wait_for can be also just a css selector
    # wait_for = "article.tease-card:nth-child(10)"

    async with AsyncWebCrawler(verbose=True) as crawler:
        js_code = [
            "const loadMoreButton = Array.from(document.querySelectorAll('button')).find(button => button.textContent.includes('Load More')); loadMoreButton && loadMoreButton.click();"
        ]
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            js_code=js_code,
            # wait_for=wait_for,
            bypass_cache=True,
        )
        print(result.markdown[:500])  # Print first 500 characters


asyncio.run(crawl_dynamic_content())

[INIT].... → Crawl4AI 0.4.1


/tmp/ipykernel_7114/2200669308.py:14: DeprecationWarning: Cache control boolean flags are deprecated and will be removed in version X.X.X. Use 'cache_mode' parameter instead. Examples:
- For bypass_cache=True, use cache_mode=CacheMode.BYPASS
- For disable_cache=True, use cache_mode=CacheMode.DISABLED
- For no_cache_read=True, use cache_mode=CacheMode.WRITE_ONLY
- For no_cache_write=True, use cache_mode=CacheMode.READ_ONLY
Pass warning=False to suppress this warning.
  result = await crawler.arun(


[FETCH]... ↓ https://www.nbcnews.com/business... | Status: True | Time: 9.90s
[SCRAPE].. ◆ Processed https://www.nbcnews.com/business... | Time: 85ms
[COMPLETE] ● https://www.nbcnews.com/business... | Status: True | Total: 9.99s
IE 11 is not supported. For an optimal experience visit our site on another browser.

Skip to Content

[NBC News Logo](https://www.nbcnews.com)

Sponsored By

  * [Live: CEO Killing](https://www.nbcnews.com/news/us-news/live-blog/live-updates-man-questioned-unitedhealthcare-ceo-brian-thompsons-shoot-rcna183160)
  * [U.S. News](https://www.nbcnews.com/us-news)
  * Local
  * [New York](https://www.nbcnews.com/new-york)
  * [Los Angeles](https://www.nbcnews.com/los-angeles)
  * [Chicago](https://ww


In [9]:
async def clean_content():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://janineintheworld.com/places-to-visit-in-central-mexico",
            excluded_tags=["nav", "footer", "aside"],
            remove_overlay_elements=True,
            word_count_threshold=10,
            bypass_cache=True,
        )
        full_markdown_length = len(result.markdown)
        fit_markdown_length = len(result.fit_markdown)
        print(f"Full Markdown Length: {full_markdown_length}")
        print(f"Fit Markdown Length: {fit_markdown_length}")
        print(result.fit_markdown[:1000])


asyncio.run(clean_content())

/tmp/ipykernel_7114/4117242193.py:3: DeprecationWarning: Cache control boolean flags are deprecated and will be removed in version X.X.X. Use 'cache_mode' parameter instead. Examples:
- For bypass_cache=True, use cache_mode=CacheMode.BYPASS
- For disable_cache=True, use cache_mode=CacheMode.DISABLED
- For no_cache_read=True, use cache_mode=CacheMode.WRITE_ONLY
- For no_cache_write=True, use cache_mode=CacheMode.READ_ONLY
Pass warning=False to suppress this warning.
  result = await crawler.arun(


TypeError: object of type 'NoneType' has no len()

In [ ]:
async def link_analysis():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            bypass_cache=True,
            exclude_external_links=True,
            exclude_social_media_links=True,
            # exclude_domains=["facebook.com", "twitter.com"]
        )
        print(f"Found {len(result.links['internal'])} internal links")
        print(f"Found {len(result.links['external'])} external links")

        for link in result.links["internal"][:5]:
            print(f"Href: {link['href']}\nText: {link['text']}\n")


asyncio.run(link_analysis())

In [ ]:
async def media_handling():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            bypass_cache=True,
            exclude_external_images=False,
            screenshot=True,
        )
        for img in result.media["images"][:5]:
            print(f"Image URL: {img['src']}, Alt: {img['alt']}, Score: {img['score']}")


asyncio.run(media_handling())

In [ ]:
async def custom_hook_workflow():
    async with AsyncWebCrawler() as crawler:
        # Set a 'before_goto' hook to run custom code just before navigation
        crawler.crawler_strategy.set_hook(
            "before_goto", lambda page: print("[Hook] Preparing to navigate...")
        )

        # Perform the crawl operation
        result = await crawler.arun(url="https://crawl4ai.com", bypass_cache=True)
        print(result.markdown[:500])  # Display the first 500 characters


asyncio.run(custom_hook_workflow())